<a href="https://colab.research.google.com/github/TejasriVaitla/Cloud-Computing/blob/main/Machine%20Learning/Movie%20Recommendation%20System/Project2/MovieLens_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import libraries

In [ ]:
#https://grouplens.org/datasets/movielens/

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=47d56870f9f208c935b6dd02db1748b0d93ca856a9f42a2b0cdbedbd25b8c3ea
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
import pandas as pd
from pyspark.sql.functions import col, explode
from pyspark import SparkContext

### Initiate spark session

In [ ]:
from pyspark.sql import SparkSession
sc = SparkContext
# sc.setCheckpointDir('checkpoint')
spark = SparkSession.builder.appName('Recommendations').getOrCreate()

# 1. Load data

In [ ]:
movies = spark.read.csv("movies.csv",header=True)
ratings = spark.read.csv("ratings.csv",header=True)

In [ ]:
ratings.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



In [ ]:
ratings.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [ ]:
ratings = ratings.\
    withColumn('userId', col('userId').cast('integer')).\
    withColumn('movieId', col('movieId').cast('integer')).\
    withColumn('rating', col('rating').cast('float')).\
    drop('timestamp')
ratings.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
|     1|     70|   3.0|
|     1|    101|   5.0|
|     1|    110|   4.0|
|     1|    151|   5.0|
|     1|    157|   5.0|
|     1|    163|   5.0|
|     1|    216|   5.0|
|     1|    223|   3.0|
|     1|    231|   5.0|
|     1|    235|   4.0|
|     1|    260|   5.0|
|     1|    296|   3.0|
|     1|    316|   3.0|
|     1|    333|   5.0|
|     1|    349|   4.0|
+------+-------+------+
only showing top 20 rows



## Calculate sparsity

In [ ]:
# Count the total number of ratings in the dataset
numerator = ratings.select("rating").count()

# Count the number of distinct userIds and distinct movieIds
num_users = ratings.select("userId").distinct().count()
num_movies = ratings.select("movieId").distinct().count()

# Set the denominator equal to the number of users multiplied by the number of movies
denominator = num_users * num_movies

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is  98.30% empty.


## Interpret ratings

In [ ]:
# Group data by userId, count ratings
userId_ratings = ratings.groupBy("userId").count().orderBy('count', ascending=False)
userId_ratings.show()

+------+-----+
|userId|count|
+------+-----+
|   414| 2698|
|   599| 2478|
|   474| 2108|
|   448| 1864|
|   274| 1346|
|   610| 1302|
|    68| 1260|
|   380| 1218|
|   606| 1115|
|   288| 1055|
|   249| 1046|
|   387| 1027|
|   182|  977|
|   307|  975|
|   603|  943|
|   298|  939|
|   177|  904|
|   318|  879|
|   232|  862|
|   480|  836|
+------+-----+
only showing top 20 rows



In [ ]:
# Group data by userId, count ratings
movieId_ratings = ratings.groupBy("movieId").count().orderBy('count', ascending=False)
movieId_ratings.show()

+-------+-----+
|movieId|count|
+-------+-----+
|    356|  329|
|    318|  317|
|    296|  307|
|    593|  279|
|   2571|  278|
|    260|  251|
|    480|  238|
|    110|  237|
|    589|  224|
|    527|  220|
|   2959|  218|
|      1|  215|
|   1196|  211|
|     50|  204|
|   2858|  204|
|     47|  203|
|    780|  202|
|    150|  201|
|   1198|  200|
|   4993|  198|
+-------+-----+
only showing top 20 rows



## Build Out An ALS Model

In [ ]:
# Import the required functions
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [ ]:
# Create test and train set
(train, test) = ratings.randomSplit([0.8, 0.2], seed = 1234)

# Create ALS model
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", nonnegative = True, implicitPrefs = False, coldStartStrategy="drop")

# Confirm that a model called "als" was created
type(als)

pyspark.ml.recommendation.ALS

## Tell Spark how to tune your ALS model

In [ ]:
# Import the requisite items
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()
            #             .addGrid(als.maxIter, [5, 50, 100, 200]) \


# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
print ("Num models to be tested: ", len(param_grid))

Num models to be tested:  16


## Build your cross validation pipeline

In [ ]:
# Build cross validation using CrossValidator
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

# Confirm cv was built
print(cv)

CrossValidator_b1da3ace58b2


## Best Model and Best Model Parameters

In [ ]:
#Fit cross validator to the 'train' dataset
model = cv.fit(train)

#Extract best model from the cv model above
best_model = model.bestModel

In [ ]:

# Print best_model
print(type(best_model))

# Complete the code below to extract the ALS model parameters
print("**Best Model**")

# # Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())

# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())

# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())

<class 'pyspark.ml.recommendation.ALSModel'>
**Best Model**
  Rank: 100
  MaxIter: 10
  RegParam: 0.15


In [ ]:
# View the predictions
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

0.867999716294944


In [ ]:
test_predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   148|    356|   4.0| 3.5113072|
|   148|   4896|   4.0| 3.5106778|
|   148|   4993|   3.0|  3.443091|
|   148|   7153|   3.0| 3.4262254|
|   148|   8368|   4.0|   3.58837|
|   148|  40629|   5.0| 3.3441403|
|   148|  50872|   3.0| 3.6789117|
|   148|  60069|   4.5| 3.6821837|
|   148|  69757|   3.5| 3.3913906|
|   148|  72998|   4.0| 3.2475388|
|   148|  81847|   4.5|  3.461065|
|   148|  98491|   5.0|  3.738956|
|   148| 115617|   3.5|  3.532233|
|   148| 122886|   3.5| 3.3519826|
|   463|    296|   4.0| 4.1448135|
|   463|    527|   4.0| 3.7833629|
|   463|   2019|   4.0|  3.930145|
|   471|    527|   4.5| 3.7986026|
|   471|   6016|   4.0| 3.9728875|
|   471|   6333|   2.5| 3.2303944|
+------+-------+------+----------+
only showing top 20 rows



## Make Recommendations

In [ ]:
# Generate n Recommendations for all users
nrecommendations = best_model.recommendForAllUsers(10)
nrecommendations.limit(10).show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1|[{3379, 5.7479773...|
|     2|[{131724, 4.79826...|
|     3|[{6835, 4.85314},...|
|     4|[{3851, 4.852987}...|
|     5|[{3379, 4.575127}...|
|     6|[{33649, 4.740606...|
|     7|[{33649, 4.450504...|
|     8|[{3379, 4.6894383...|
|     9|[{3379, 4.7961397...|
|    10|[{71579, 4.531901...|
+------+--------------------+



In [ ]:
nrecommendations = nrecommendations\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('userId', col("rec_exp.movieId"), col("rec_exp.rating"))

nrecommendations.limit(15).show()

+------+-------+---------+
|userId|movieId|   rating|
+------+-------+---------+
|     1|   3379|5.7479773|
|     1|  33649| 5.524695|
|     1|   5490| 5.442981|
|     1| 171495|5.4243693|
|     1|   5416|5.3650436|
|     1|   5328|5.3650436|
|     1|   3951|5.3650436|
|     1|  78836| 5.360681|
|     1|   5915|5.3126545|
|     1| 184245|5.3088217|
|     2| 131724| 4.798265|
|     2|   3379|4.6584625|
|     2| 171495|4.4413843|
|     2|  33649|4.4194336|
|     2|  78836| 4.392709|
+------+-------+---------+



## Do the recommendations make sense?
Lets merge movie name and genres to teh recommendation matrix for interpretability.

In [ ]:
nrecommendations.join(movies, on='movieId').filter('userId = 108').show()

+-------+------+---------+--------------------+--------------------+
|movieId|userId|   rating|               title|              genres|
+-------+------+---------+--------------------+--------------------+
|   3379|   108| 5.220047| On the Beach (1959)|               Drama|
|   3925|   108| 5.000344|Stranger Than Par...|        Comedy|Drama|
|  33649|   108|4.9513683|  Saving Face (2004)|Comedy|Drama|Romance|
|   6201|   108| 4.912661|    Lady Jane (1986)|       Drama|Romance|
| 171495|   108| 4.877958|              Cosmos|  (no genres listed)|
|   3451|   108|4.8723145|Guess Who's Comin...|               Drama|
|   3795|   108| 4.852234|Five Senses, The ...|               Drama|
|  93008|   108| 4.839637|Very Potter Seque...|      Comedy|Musical|
|  77846|   108| 4.839637| 12 Angry Men (1997)|         Crime|Drama|
|  25906|   108| 4.839637|Mr. Skeffington (...|       Drama|Romance|
+-------+------+---------+--------------------+--------------------+



In [ ]:
ratings.join(movies, on='movieId').filter('userId = 108').sort('rating', ascending=False).limit(15).show()

+-------+------+------+--------------------+--------------------+
|movieId|userId|rating|               title|              genres|
+-------+------+------+--------------------+--------------------+
|   3795|   108|   5.0|Five Senses, The ...|               Drama|
|   5450|   108|   5.0|Lovely & Amazing ...|Comedy|Drama|Romance|
|   4027|   108|   5.0|O Brother, Where ...|Adventure|Comedy|...|
|     82|   108|   5.0|Antonia's Line (A...|        Comedy|Drama|
|   4246|   108|   5.0|Bridget Jones's D...|Comedy|Drama|Romance|
|    541|   108|   5.0| Blade Runner (1982)|Action|Sci-Fi|Thr...|
|   4308|   108|   5.0| Moulin Rouge (2001)|Drama|Musical|Rom...|
|    838|   108|   5.0|         Emma (1996)|Comedy|Drama|Romance|
|   4372|   108|   5.0|Crazy/Beautiful (...|       Drama|Romance|
|   1204|   108|   5.0|Lawrence of Arabi...| Adventure|Drama|War|
|   4878|   108|   5.0| Donnie Darko (2001)|Drama|Mystery|Sci...|
|   1684|   108|   5.0|Mrs. Dalloway (1997)|       Drama|Romance|
|   4973| 